# Scrapping de maneira simples
O objetivo desse notebook é mostrar como fazer scrapping de maneira simples, sem fazer uso de bibliotecas sofisticadas como Selenium, BeautifulSoup, etc.

A ideia é que devemos sempre procurar por APIs ocultas nos sites que pesquisamos. Essas APIs são usadas internamente pelos sites para recuperar dados que são exibidos na tela. Detectar e fazer uso dessas APIs facilita o processo de scrapping e deve ser sempre a primeira opção do engenheiro de dados. 

## Primeiro Exemplo

Fazer scrapping dos ativos listados em balcão da bolsa de valores.
- A página inicial de consulta é https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/

### Primeiro passo
- Abrir o console de desenvolvedor no seu navegador (No Firefox pode ser acessado pela tecla F12 ou apertando Ctrl + Shift + I)
- Clique em Rede e aperte F5 para recarregar a página. Observe uma série de requisições GET e POST no console.
- Procure por uma requisição GET do tipo json
- Verifique o conteúdo de resposta dessa requisição. Para isso, clique na requisição e depois clique em resposta.
- Verifique se a resposta é aquilo que procura. Em caso positivo, parabéns, você identificou a chamada da API. Ela será nosso ponto de partida.
- Clique com o botão direito na requisição e clique em Copiar -> Copiar URL

In [10]:
import requests
import json

url = 'https://arquivos.b3.com.br/api/channels/34dcaaeb-0306-4f45-a83e-4f66a23b42fa/subchannels/cc188e40-03be-408e-aa86-501926b97a76/publications?&lang=pt'

page_request = requests.get(url=url)
page_response = json.loads(page_request.content)
page_response[0]

{'fileName': 'InstrumentsConsolidatedFile_20220908_1.csv',
 'extension': '.csv',
 'friendlyName': 'Cadastro de Instrumentos (Listado)',
 'fileId': '8c5166d5-44e0-4850-acae-a9bffa68411a',
 'subChannelId': 'cc188e40-03be-408e-aa86-501926b97a76',
 'dateTime': '2022-09-08T00:00:00',
 'fileDate': '2022-09-08T18:28:00'}

### Segundo passo
- Parabéns, agora você já tem acesso a todos os metadados dos arquivos que deseja extrair do site, mas esse foi só o primeiro passo. Agora é necessário fazer as requisições que farão os downloads dos arquivos de fato.
- Clique na lixeira do console para apagar todas as requisições passadas.
- Em seguida clique num arquivo qualquer para fazer download. Observe as novas requisições que o site fez.
    - Veja que dessa vez há duas novas requisições de interesse. Uma do tipo JSON e outra do tipo octet-stream (que é o arquivo binário que foi baixado)
    - Ao observar a primeira requisição, podemos ver na URL que ele faz a requisição usando 2 parâmetros: fileName e date. Nós já temos esse parâmetros do passo anterior.
    - Ao observar a segunda requisição, vemos que o parâmetro passado é um token. Esse token pode ser obtido como resposta da primeira requisição que nós fizemos.
- Vamos copiar as URLs das duas requisições que nós obtivemos. Vamos precisar delas.

In [11]:
# Requisição para obter o token
fileName = page_response[0].get('fileName').split('File')[0]
date = page_response[0].get('dateTime').split('T')[0]
url_req1 = f'https://arquivos.b3.com.br/api/download/requestname?fileName={fileName}&date={date}&recaptchaToken='
page_request_req1 = requests.get(url_req1)
page_response_req1 = json.loads(page_request_req1.content)

# Requisição para obter o arquivo
token = page_response_req1.get('token')
url_req2 = f'https://arquivos.b3.com.br/api/download/?token={token}'
page_request_req2 = requests.get(url_req2)
page_request_req2 = page_request_req2.content

### Terceiro passo
- Agora que temos o arquivo em memória, basta salvá-lo em CSV para fazer tratamento a posteriori.

In [12]:
import os
os.makedirs('data/produtos_listados', exist_ok=True)
with open(f'data/produtos_listados/{page_response[0].get("fileName")}', 'wb') as f:
    f.write(page_request_req2)

### Extra: automatizando o passo a passo anterior para extrair todos os arquivos de uma vez
- Para isso vamos precisar de um loop
- Depois é só transportar o código para uma task do airflow! :)

In [13]:
import requests
import json

url = 'https://arquivos.b3.com.br/api/channels/34dcaaeb-0306-4f45-a83e-4f66a23b42fa/subchannels/cc188e40-03be-408e-aa86-501926b97a76/publications?&lang=pt'

page_request = requests.get(url=url)
page_response = json.loads(page_request.content)

for page in page_response:
    # Requisição para obter o token
    fileName = page.get('fileName').split('File')[0]
    date = page.get('dateTime').split('T')[0]
    url_req1 = f'https://arquivos.b3.com.br/api/download/requestname?fileName={fileName}&date={date}&recaptchaToken='
    page_request_req1 = requests.get(url_req1)
    page_response_req1 = json.loads(page_request_req1.content)

    # Requisição para obter o arquivo
    token = page_response_req1.get('token')
    url_req2 = f'https://arquivos.b3.com.br/api/download/?token={token}'
    page_request_req2 = requests.get(url_req2)
    page_request_req2 = page_request_req2.content
    
    with open(f'data/produtos_listados/{page.get("fileName")}', 'wb') as f:
        f.write(page_request_req2)

## Segundo Exemplo
- Fazer scrapping de serviços oferecidos pelo IBGE
- Deseja-se baixar os dados dos perfis dos municípios
- Site inicial de consulta: https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html

### Primeiro passo
- O primeiro passo é idêntico ao executado no primeiro exemplo.
- A diferença é que os dados do IBGE estão dispostos numa estruturas de pastas, como observaremos a seguir.

In [5]:
import requests
import json
import os

url = f'https://servicodados.ibge.gov.br/api/v1/downloads/estatisticas?caminho=/&nivel=1'

page_request = requests.get(url)
page_response = json.loads(page_request.content)


# procurando pelo elemento com os dados sobre os municipios
for response in page_response:
    if response.get('name') == 'Perfil_Municipios':
        # criando pasta raiz de dados
        os.makedirs('data/Perfil_Municipios', exist_ok=True)
        meta_mun = response
        break
meta_mun

{'name': 'Perfil_Municipios',
 'isFolder': True,
 'url': None,
 'path': 'Perfil_Municipios',
 'fileSize': 0,
 'children': [{'name': '2001',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Municipios/2001',
   'fileSize': 0,
   'children': None},
  {'name': '2002',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Municipios/2002',
   'fileSize': 0,
   'children': None},
  {'name': '2004',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Municipios/2004',
   'fileSize': 0,
   'children': None},
  {'name': '2005',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Municipios/2005',
   'fileSize': 0,
   'children': None},
  {'name': '2006',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Municipios/2006',
   'fileSize': 0,
   'children': None},
  {'name': '2008',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Municipios/2008',
   'fileSize': 0,
   'children': None},
  {'name': '2009',
   'isFolder': True,
   'url': None,
   'path': 'Perfil_Munici

### Segundo passo
- Observamos no print acima que as chaves retornadas acima tem três parâmetros de interesse:
    - isFolder: se o valor for True, indica que esse JSON é apenas uma pasta. Ou seja, os arquivos para serem baixados noutro nível do JSON.
    - path: indica o parâmetro que devemos passar na API. Ao passar o path em '?caminho=' vamos recuperar outro JSON que pode conter uma URL com o arquivo a ser baixado.
    - children: indica outros nós aninhados em nosso JSON. Nesse caso nós temos as subpastas relacionadas a pasta principal 'Perfil_Municipios'. Vamos começar nossa busca por aí. 
- Vamos fazer um teste com o primeiro elemento desse JSON pra ver se a gente acha a url pra fazer download do arquivo

In [6]:
children = meta_mun.get('children')
caminho = children[0].get('path')
url = url = f'https://servicodados.ibge.gov.br/api/v1/downloads/estatisticas?caminho={caminho}/&nivel=1'
page_request = requests.get(url)
page_response = page_request.content
page_response

b'[{"name":"Tabelas_2001.zip","isFolder":false,"url":"https://ftp.ibge.gov.br/Perfil_Municipios/2001/Tabelas_2001.zip","path":null,"fileSize":364147,"children":null}]'

### Terceiro passo
- Achamos nossa primeira URL com um arquivo que queremos extrair. Aqui temos três parâmetros de interesse:
    - name: nome do arquivo que será baixado.
    - url: a url com o nosso arquivo. Note que ela aponta para um servidor FTP, ou seja, de transferência de dados.
    - isFolder: quando essa variável é false, indica que chegamos na raiz das pastas, ou seja, onde os arquivos se encontram de fato.
    - Nesse ponto já poderíamos fazer nosso primeiro download, mas precisamos tratar algumas situações antes de automatizar tudo. Isso será discutido no próximo passo.

### Quarto passo
- A nossa solução precisa tratar o seguinte cenário: como extrair urls que estão dentro subpastas?
    - Ex: Perfil_Municipios/2008/pdf/1_bimestre/parte_2/arquivo.txt
- Para isso, foi necessário construir uma subrotina que vai navegando dentro de subpastas até chegar na raiz, onde encontram-se os arquivos.

In [7]:
# Função privada que navega por subpastas até que urls com arquivos sejam encontradas.
def get_all_files(json_doc):
    # Se o JSON for uma pasta, ele criará essa pasta localmente e chamará essa 
    # função novamente para procurar por subpastas dentro dessa pasta
    if json_doc.get('isFolder'):
        caminho = json_doc.get('path')
        url = f'https://servicodados.ibge.gov.br/api/v1/downloads/estatisticas?caminho={caminho}/&nivel=1'
        os.makedirs(f'data/{caminho}', exist_ok=True)
        page_request = requests.get(url)
        page_response = json.loads(page_request.content)
        # A response pode vir em formato de lista (contém várias subpastas). Para lidar com isso é necessário um loop
        # Ex: [Perfil_Municipios/2008/xls/, Perfil_Municipios/2008/csv]
        for response in page_response:
            # Se não for folder, quer dizer que chegamos à raiz. 
            # Vamos extrair o arquivo e salvá-lo na pasta que criamos localmente.
            if not response.get('isFolder'):
                file = requests.get(response.get('url'))
                filename = response.get('name')
                with open(f'data/{caminho}/{filename}', 'wb') as f:
                    f.write(file.content)
            # Caso contrário, vamos chamar essa função novamente.
            # Dessa vez vamos passar como parâmetro a próxima subpasta.
            # Ex: Se estamos em Perfil_Municipios/2008/ vamos chamar Perfil_Municipios/2008/xls/
            else:
                get_all_files(response)
    # Caso contrário, vamos chamar essa função novamente.
    # Dessa vez vamos passar como parâmetro a próxima subpasta.
    # Ex: Se estamos em Perfil_Municipios/2008/ vamos chamar Perfil_Municipios/2008/xls/
    else:
        file = requests.get(json_doc.get('url'))
        filename = json_doc.get('name')
        with open(f'data/{caminho}/{filename}', 'wb') as f:
            f.write(file.content)

### Quinto passo
- Por fim, executamos a subrotina criada acima para todas as pastas recuperadas em Perfil_Municipios/

In [8]:
childrens = meta_mun.get('children')
for children in childrens:
    get_all_files(children)